In [1]:
import os
import datetime as dt
import pandas as pd
import numpy as np
import time

main_dir = os.getcwd()
os.chdir(main_dir + '\\Leaps Pulling')
cad_dir = main_dir + '\\Historical Queries\\CAD'
usd_dir = main_dir + '\\Historical Queries\\US'

from helpers import *
from yahoo_query import *

os.chdir(main_dir)

In [91]:
def create_dgi(annual_data):
    dgi_data = annual_data[['totalLiab',
                             'totalStockholderEquity',
                             'longTermInvestments',
                             'shortTermInvestments',
                             'netIncome','operatingIncome',
                             'totalRevenue','dividendsPaid',
                             'investments','Underlying']]
    dgi_data['dividendsPaid'] = -dgi_data['dividendsPaid']

    dgi_data['payoutRatio'] = dgi_data.dividendsPaid/dgi_data.netIncome
    dgi_data['debtEquityRatio'] = dgi_data.totalLiab/dgi_data.totalStockholderEquity
    dgi_data['netMargin'] = dgi_data.netIncome/dgi_data.totalRevenue
    dgi_data['roic'] = dgi_data.operatingIncome/(dgi_data.longTermInvestments + dgi_data.shortTermInvestments)

    dgi_data = dgi_data[['Underlying','payoutRatio','debtEquityRatio','netMargin','roic','dividendsPaid']]
    return dgi_data

# DGI Portion

def dgi_scores(annual_df,keystats, min_dgi_score):
    dgi_df = create_dgi(annual_df).fillna(0).replace(-np.inf,0).replace(np.inf,0)

    dgi_df_scores = pd.DataFrame(columns = ['divGrowth','payoutChange','payoutRatio',
                                             'debtEquityRatio','netMargin','roic'],
                                  index = dgi_df.Underlying.drop_duplicates().tolist()).fillna(0)

    for ticker in dgi_df.Underlying.drop_duplicates():
        curr_dgi = dgi_df[dgi_df.Underlying == ticker].sort_index()
        if sum(curr_dgi['dividendsPaid'].pct_change() >= 0.02) == len(curr_dgi) -1:
            dgi_df_scores.loc[ticker, 'divGrowth'] = 1
        if sum(curr_dgi.payoutRatio.pct_change() <= 1) == len(curr_dgi) - 1:
            dgi_df_scores.loc[ticker, 'payoutChange'] = 1
        if curr_dgi.payoutRatio.tail(3).mean() <= 0.6:
            dgi_df_scores.loc[ticker, 'payoutRatio'] = 1
        if sum(curr_dgi.debtEquityRatio.tail(3) <= 1.5) == len(curr_dgi) - 1:
            dgi_df_scores.loc[ticker, 'debtEquityRatio'] = 1
        if sum(curr_dgi.netMargin.tail(3) >= 0.07) == len(curr_dgi) - 1:
            dgi_df_scores.loc[ticker, 'netMargin'] = 1
        if sum(curr_dgi.roic.tail(3) >= 0.2) == len(curr_dgi) - 1:
            dgi_df_scores.loc[ticker, 'roic'] = 1

    dgi_df_scores['score'] = dgi_df_scores.sum(axis = 1)
    dgi_df_scores = dgi_df_scores[dgi_df_scores.score >= min_dgi_score].join(keystats[['currentPrice','returnOnAssets','returnOnEquity']])

    dgi_df_scores = dgi_df_scores.sort_values('returnOnAssets', ascending = False)
    dgi_df_scores['roaRank'] = range(1,len(dgi_df_scores) + 1)
    dgi_df_scores = dgi_df_scores.sort_values('returnOnEquity', ascending = False)
    dgi_df_scores['roeRank'] = range(1,len(dgi_df_scores) + 1)
    dgi_df_scores['roRank'] = dgi_df_scores.roaRank + dgi_df_scores.roeRank
    dgi_df_scores = dgi_df_scores.sort_values('roRank')
    return dgi_df_scores[['currentPrice','score','roRank']]

# filterd_dgi = dgi_df[(dgi_df['debtEquityAvg'] <= 1.5) &
#        (dgi_df['divGrowth'] >= 0.03) &
#        (dgi_df['payoutChange'] <= 0) &
#        (dgi_df['payoutAvg'] <= 0.6) & (dgi_df['payoutAvg'] >= 0) &
#        (dgi_df['netMarginAvg'] >= 0.07) &
#        (dgi_df['roicAvg'] >= 0.2)].sort_values('divGrowth', ascending = False)

# DGI Filter

In [3]:
os.chdir(cad_dir)

file_date = '2018-09-18'
cad_annual = pd.read_csv('cad_annual_{}.csv'.format(file_date), index_col = 0)
cad_keystats = pd.read_csv('cad_keystats_{}.csv'.format(file_date), index_col = 0)
cad_quarterly = pd.read_csv('cad_quarterly_{}.csv'.format(file_date), index_col = 0)

os.chdir(main_dir)

os.chdir(usd_dir)

file_date = '2018-09-17'
us_annual = pd.read_csv('us_annual_{}.csv'.format(file_date), index_col = 0)
us_keystats = pd.read_csv('us_keystats_{}.csv'.format(file_date), index_col = 0)
us_quarterly = pd.read_csv('us_quarterly_{}.csv'.format(file_date), index_col = 0)

os.chdir(main_dir)

In [96]:
cad_dgi_scores = dgi_scores(cad_annual, cad_keystats, 4)
print("CAD")
cad_dgi_scores.head(20)

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

CAD


,currentPrice,score,roRank
TCN.TO,11.17,4,8
BAD.TO,28.45,4,10
WTE.TO,26.56,4,11
CNR.TO,113.95,5,12
LUC.TO,2.31,4,12
MG.TO,69.86,4,20
ENGH.TO,80.15,5,21
TIH.TO,64.41,4,21
MRU.TO,41.07,4,22
RCH.TO,30.48,5,24


In [97]:
us_dgi_score = dgi_scores(us_annual, us_keystats, 5)
print("USD")
us_dgi_score.head(20)

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

USD


,currentPrice,score,roRank
MA,216.47,5,4
INTU,225.47,5,7
TXN,103.70,6,8
SBUX,54.57,5,12
AMAT,38.70,5,13
ACN,171.04,5,15
MKTX,179.79,5,19
ROL,63.26,5,22
AAPL,217.88,5,24
SEIC,62.47,5,24


# LEAPs Filtering